In [3]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Display settings
pd.set_option('display.max_columns', None)

print("✅ Setup complete!")

✅ Setup complete!


In [4]:
# Connect to DuckDB and load data
conn = duckdb.connect()

conn.execute("""
    CREATE TABLE heart AS 
    SELECT * FROM read_csv_auto('../data/raw/tabular/heart_disease_uci.csv')
""")

conn.execute("""
    CREATE TABLE diabetes AS 
    SELECT * FROM read_csv_auto('../data/raw/tabular/diabetes_prediction_dataset.csv')
""")

# Load into pandas DataFrames
heart_df = conn.execute("SELECT * FROM heart").fetchdf()
diabetes_df = conn.execute("SELECT * FROM diabetes").fetchdf()

print(f"✅ Heart: {heart_df.shape[0]} rows, {heart_df.shape[1]} columns")
print(f"✅ Diabetes: {diabetes_df.shape[0]} rows, {diabetes_df.shape[1]} columns")

✅ Heart: 920 rows, 16 columns
✅ Diabetes: 100000 rows, 9 columns


In [5]:
# Look at current state of heart data
print("=== Heart Dataset - Before Feature Engineering ===")
print(f"\nColumns: {heart_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
heart_df.head()

=== Heart Dataset - Before Feature Engineering ===

Columns: ['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']

First 5 rows:


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145,233,True,lv hypertrophy,150,False,2.3,downsloping,0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160,286,False,lv hypertrophy,108,True,1.5,flat,3,normal,2
2,3,67,Male,Cleveland,asymptomatic,120,229,False,lv hypertrophy,129,True,2.6,flat,2,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130,250,False,normal,187,False,3.5,downsloping,0,normal,0
4,5,41,Female,Cleveland,atypical angina,130,204,False,lv hypertrophy,172,False,1.4,upsloping,0,normal,0


In [6]:
# Check the messy 'sex' column
print("=== Sex Column - Data Quality Issue ===")
print(f"\nUnique values: {heart_df['sex'].unique()}")
print(f"\nValue counts:")
print(heart_df['sex'].value_counts())

=== Sex Column - Data Quality Issue ===

Unique values: ['Male' 'Female']

Value counts:
sex
Male      726
Female    194
Name: count, dtype: int64


In [7]:
# Fix the messy sex column - standardize to 0 and 1
def clean_sex_column(value):
    """Convert all sex values to 0 (Female) or 1 (Male)"""
    if value in ['Male', '1', 1]:
        return 1
    elif value in ['Female', '0', 0]:
        return 0
    else:
        return None  # Unknown values

# Apply the fix
heart_df['sex_clean'] = heart_df['sex'].apply(clean_sex_column)

# Verify the fix
print("=== After Cleaning ===")
print(f"Unique values: {heart_df['sex_clean'].unique()}")
print(f"\nValue counts:")
print(heart_df['sex_clean'].value_counts())

=== After Cleaning ===
Unique values: [1 0]

Value counts:
sex_clean
1    726
0    194
Name: count, dtype: int64


In [8]:


# Create age groups
def get_age_group(age):
    if age < 40:
        return 'Young'
    elif age < 60:
        return 'Middle'
    else:
        return 'Senior'

heart_df['age_group'] = heart_df['age'].apply(get_age_group)

print("✅ age_group created!")
print(heart_df['age_group'].value_counts())

✅ age_group created!
age_group
Middle    587
Senior    253
Young      80
Name: count, dtype: int64


In [9]:
# Create age decade (20s, 30s, 40s, etc.)
heart_df['age_decade'] = (heart_df['age'] // 10) * 10

print("✅ age_decade created!")
print(heart_df['age_decade'].value_counts().sort_index())

✅ age_decade created!
age_decade
20      4
30     76
40    212
50    375
60    222
70     31
Name: count, dtype: int64


In [10]:
# Create cholesterol categories
def get_chol_category(chol):
    if chol == 0:
        return 'Missing'
    elif chol < 200:
        return 'Normal'
    elif chol < 240:
        return 'Borderline'
    else:
        return 'High'

heart_df['chol_category'] = heart_df['chol'].apply(get_chol_category)

print("✅ chol_category created!")
print(heart_df['chol_category'].value_counts())

✅ chol_category created!
chol_category
High          389
Borderline    231
Missing       172
Normal        128
Name: count, dtype: int64


In [11]:
# Create blood pressure categories
def get_bp_category(bp):
    if bp == 0:
        return 'Missing'
    elif bp < 120:
        return 'Normal'
    elif bp < 140:
        return 'Elevated'
    else:
        return 'High'

heart_df['bp_category'] = heart_df['trestbps'].apply(get_bp_category)

print("✅ bp_category created!")
print(heart_df['bp_category'].value_counts())

✅ bp_category created!
bp_category
Elevated    388
High        370
Normal      161
Missing       1
Name: count, dtype: int64


In [15]:
# Create a flag for missing cholesterol (0 values or NA)
heart_df['chol_missing'] = ((heart_df['chol'] == 0) | (heart_df['chol'].isna())).astype(int)

print("✅ chol_missing created!")
print(heart_df['chol_missing'].value_counts())

✅ chol_missing created!
chol_missing
0    718
1    202
Name: count, dtype: int64


In [16]:
# Step 1: Calculate median of non-zero values
chol_median = heart_df[heart_df['chol'] > 0]['chol'].median()

# Step 2: Create function to fill zeros
def fill_zero_with_median(value):
    if value == 0:
        return chol_median
    else:
        return value

# Step 3: Apply the function
heart_df['chol_filled'] = heart_df['chol'].apply(fill_zero_with_median)

print(f"✅ chol_filled created!")
print(f"Median cholesterol: {chol_median}")
print(f"Zero values before: {(heart_df['chol'] == 0).sum()}")
print(f"Zero values after: {(heart_df['chol_filled'] == 0).sum()}")

✅ chol_filled created!
Median cholesterol: 239.5
Zero values before: 172
Zero values after: 0


In [17]:
# Create cholesterol to age ratio
def chol_to_age_ratio(row):
    if row['age'] > 0:
        return row['chol_filled'] / row['age']
    else:
        return 0

heart_df['chol_to_age'] = heart_df.apply(chol_to_age_ratio, axis=1)

print("✅ chol_to_age created!")
print(f"Average ratio: {heart_df['chol_to_age'].mean():.2f}")
print(f"Min: {heart_df['chol_to_age'].min():.2f}")
print(f"Max: {heart_df['chol_to_age'].max():.2f}")

✅ chol_to_age created!
Average ratio: 4.72
Min: 1.52
Max: 16.53


In [18]:
def bp_to_age_ratio(row):
    if row['age'] > 0:
        return row['trestbps'] / row['age']
    else:
        return 0

heart_df['bp_to_age'] = heart_df.apply(bp_to_age_ratio, axis=1)

print("✅ bp_to_age created!")
print(f"Average ratio: {heart_df['bp_to_age'].mean():.2f}")

✅ bp_to_age created!
Average ratio: 2.56


In [19]:
# Create elderly flag (age > 60)
def is_elderly(age):
    if age > 60:
        return 1
    else:
        return 0

heart_df['is_elderly'] = heart_df['age'].apply(is_elderly)

print("✅ is_elderly created!")
print(heart_df['is_elderly'].value_counts())

✅ is_elderly created!
is_elderly
0    699
1    221
Name: count, dtype: int64


In [20]:
# Create high cholesterol flag (chol > 240)
def high_chol_flag(chol):
    if chol > 240:
        return 1
    else:
        return 0

heart_df['high_chol'] = heart_df['chol_filled'].apply(high_chol_flag)

print("✅ high_chol created!")
print(heart_df['high_chol'].value_counts())

✅ high_chol created!
high_chol
0    569
1    351
Name: count, dtype: int64


In [21]:
# Create high blood pressure flag (bp > 140)
def high_bp_flag(bp):
    if bp > 140:
        return 1
    else:
        return 0

heart_df['high_bp'] = heart_df['trestbps'].apply(high_bp_flag)

print("✅ high_bp created!")
print(heart_df['high_bp'].value_counts())

✅ high_bp created!
high_bp
0    711
1    209
Name: count, dtype: int64


In [23]:
# Create a combined risk score - handles missing values
def calculate_risk_score(row):
    score = 0
    
    # Age risk
    if pd.notna(row['age']) and row['age'] > 60:
        score += 1
    
    # Cholesterol risk
    if pd.notna(row['chol_filled']) and row['chol_filled'] > 240:
        score += 1
    
    # Blood pressure risk
    if pd.notna(row['trestbps']) and row['trestbps'] > 140:
        score += 1
    
    # Sex risk (males have higher risk)
    if pd.notna(row['sex_clean']) and row['sex_clean'] == 1:
        score += 1
    
    return score

heart_df['risk_score'] = heart_df.apply(calculate_risk_score, axis=1)

print("✅ risk_score created!")
print(heart_df['risk_score'].value_counts().sort_index())

✅ risk_score created!
risk_score
0     63
1    386
2    308
3    147
4     16
Name: count, dtype: int64


In [24]:
# Create binary target (0 = no disease, 1 = has disease)
def make_binary_target(num):
    if pd.notna(num) and num > 0:
        return 1
    else:
        return 0

heart_df['target_binary'] = heart_df['num'].apply(make_binary_target)

print("✅ target_binary created!")
print(heart_df['target_binary'].value_counts())

✅ target_binary created!
target_binary
1    509
0    411
Name: count, dtype: int64


In [25]:
def heart_rate_reserve(row):
    if pd.notna(row['thalch']) and pd.notna(row['trestbps']):
        return row['thalch'] - row['trestbps']
    else:
        return 0

heart_df['hr_reserve'] = heart_df.apply(heart_rate_reserve, axis=1)

print("✅ hr_reserve created!")
print(f"Average HR Reserve: {heart_df['hr_reserve'].mean():.2f}")

✅ hr_reserve created!
Average HR Reserve: 5.16


In [26]:
# See all columns we have now
print("=== Heart Dataset - All Columns ===")
print(f"Original columns: 16")
print(f"Total columns now: {len(heart_df.columns)}")
print(f"\nNew features created: {len(heart_df.columns) - 16}")

print("\n=== All Columns ===")
for col in heart_df.columns:
    print(f"  - {col}")

=== Heart Dataset - All Columns ===
Original columns: 16
Total columns now: 31

New features created: 15

=== All Columns ===
  - id
  - age
  - sex
  - dataset
  - cp
  - trestbps
  - chol
  - fbs
  - restecg
  - thalch
  - exang
  - oldpeak
  - slope
  - ca
  - thal
  - num
  - sex_clean
  - age_group
  - age_decade
  - chol_category
  - bp_category
  - chol_missing
  - chol_filled
  - chol_to_age
  - bp_to_age
  - is_elderly
  - high_chol
  - high_bp
  - risk_score
  - target_binary
  - hr_reserve


In [27]:
# Look at diabetes dataset
print("=== Diabetes Dataset ===")
print(f"Shape: {diabetes_df.shape}")
print(f"\nColumns: {diabetes_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
diabetes_df.head()

=== Diabetes Dataset ===
Shape: (100000, 9)

Columns: ['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes']

First 5 rows:


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [28]:
# Encode gender (Male = 1, Female = 0)
def encode_gender(gender):
    if gender == 'Male':
        return 1
    elif gender == 'Female':
        return 0
    else:
        return None

diabetes_df['gender_encoded'] = diabetes_df['gender'].apply(encode_gender)

print("✅ gender_encoded created!")
print(diabetes_df['gender_encoded'].value_counts())

✅ gender_encoded created!
gender_encoded
0.0    58552
1.0    41430
Name: count, dtype: int64


In [29]:
# Create age groups for diabetes dataset
def get_age_group(age):
    if age < 40:
        return 'Young'
    elif age < 60:
        return 'Middle'
    else:
        return 'Senior'

diabetes_df['age_group'] = diabetes_df['age'].apply(get_age_group)

print("✅ age_group created!")
print(diabetes_df['age_group'].value_counts())

✅ age_group created!
age_group
Young     45487
Middle    29458
Senior    25055
Name: count, dtype: int64


In [30]:
# Create BMI categories
def get_bmi_category(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal'
    elif bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

diabetes_df['bmi_category'] = diabetes_df['bmi'].apply(get_bmi_category)

print("✅ bmi_category created!")
print(diabetes_df['bmi_category'].value_counts())

✅ bmi_category created!
bmi_category
Overweight     45751
Obese          23536
Normal         22219
Underweight     8494
Name: count, dtype: int64


In [31]:
# Create blood glucose categories
def get_glucose_category(glucose):
    if glucose < 100:
        return 'Normal'
    elif glucose < 126:
        return 'Prediabetes'
    else:
        return 'Diabetes'

diabetes_df['glucose_category'] = diabetes_df['blood_glucose_level'].apply(get_glucose_category)

print("✅ glucose_category created!")
print(diabetes_df['glucose_category'].value_counts())

✅ glucose_category created!
glucose_category
Diabetes       71856
Normal         21119
Prediabetes     7025
Name: count, dtype: int64


In [32]:
def get_hba1c_category(hba1c):
    if hba1c < 5.7:
        return 'Normal'
    elif hba1c < 6.5:
        return 'Prediabetes'
    else:
        return 'Diabetes'

diabetes_df['hba1c_category'] = diabetes_df['HbA1c_level'].apply(get_hba1c_category)

print("✅ hba1c_category created!")
print(diabetes_df['hba1c_category'].value_counts())

✅ hba1c_category created!
hba1c_category
Prediabetes    41346
Normal         37857
Diabetes       20797
Name: count, dtype: int64


In [33]:
# Encode smoking history to numbers
def encode_smoking(smoking):
    if smoking == 'never':
        return 0
    elif smoking == 'former':
        return 1
    elif smoking == 'current':
        return 2
    elif smoking == 'not current':
        return 1
    elif smoking == 'ever':
        return 1
    else:
        return 0  # No Info or unknown

diabetes_df['smoking_encoded'] = diabetes_df['smoking_history'].apply(encode_smoking)

print("✅ smoking_encoded created!")
print(diabetes_df['smoking_encoded'].value_counts())

✅ smoking_encoded created!
smoking_encoded
0    70911
1    19803
2     9286
Name: count, dtype: int64


In [34]:
# Create obese flag (BMI > 30)
def is_obese(bmi):
    if bmi > 30:
        return 1
    else:
        return 0

diabetes_df['is_obese'] = diabetes_df['bmi'].apply(is_obese)

print("✅ is_obese created!")
print(diabetes_df['is_obese'].value_counts())

✅ is_obese created!
is_obese
0    76517
1    23483
Name: count, dtype: int64


In [35]:
# Create elderly flag (age > 60)
def is_elderly(age):
    if age > 60:
        return 1
    else:
        return 0

diabetes_df['is_elderly'] = diabetes_df['age'].apply(is_elderly)

print("✅ is_elderly created!")
print(diabetes_df['is_elderly'].value_counts())

✅ is_elderly created!
is_elderly
0    76368
1    23632
Name: count, dtype: int64


In [36]:
# Create high glucose flag (glucose > 140)
def high_glucose_flag(glucose):
    if glucose > 140:
        return 1
    else:
        return 0

diabetes_df['high_glucose'] = diabetes_df['blood_glucose_level'].apply(high_glucose_flag)

print("✅ high_glucose created!")
print(diabetes_df['high_glucose'].value_counts())

✅ high_glucose created!
high_glucose
0    51372
1    48628
Name: count, dtype: int64


In [37]:
# Create age * BMI interaction feature
diabetes_df['age_bmi'] = diabetes_df['age'] * diabetes_df['bmi']

print("✅ age_bmi created!")
print(f"Average: {diabetes_df['age_bmi'].mean():.2f}")
print(f"Min: {diabetes_df['age_bmi'].min():.2f}")
print(f"Max: {diabetes_df['age_bmi'].max():.2f}")

✅ age_bmi created!
Average: 1194.77
Min: 0.88
Max: 7100.80


In [38]:
# Create glucose * BMI interaction feature
diabetes_df['glucose_bmi'] = diabetes_df['blood_glucose_level'] * diabetes_df['bmi']

print("✅ glucose_bmi created!")
print(f"Average: {diabetes_df['glucose_bmi'].mean():.2f}")
print(f"Min: {diabetes_df['glucose_bmi'].min():.2f}")
print(f"Max: {diabetes_df['glucose_bmi'].max():.2f}")

✅ glucose_bmi created!
Average: 3796.51
Min: 866.15
Max: 26616.00


In [39]:
# Create combined risk score for diabetes
def diabetes_risk_score(row):
    score = 0
    
    # Age risk
    if pd.notna(row['age']) and row['age'] > 60:
        score += 1
    
    # BMI risk
    if pd.notna(row['bmi']) and row['bmi'] > 30:
        score += 1
    
    # Glucose risk
    if pd.notna(row['blood_glucose_level']) and row['blood_glucose_level'] > 140:
        score += 1
    
    # HbA1c risk
    if pd.notna(row['HbA1c_level']) and row['HbA1c_level'] > 6.5:
        score += 1
    
    # Smoking risk
    if pd.notna(row['smoking_encoded']) and row['smoking_encoded'] == 2:
        score += 1
    
    return score

diabetes_df['risk_score'] = diabetes_df.apply(diabetes_risk_score, axis=1)

print("✅ risk_score created!")
print(diabetes_df['risk_score'].value_counts().sort_index())

✅ risk_score created!
risk_score
0    25914
1    41439
2    23492
3     7635
4     1464
5       56
Name: count, dtype: int64


In [40]:
# Create combined hypertension and heart disease flag
def has_both_conditions(row):
    if row['hypertension'] == 1 and row['heart_disease'] == 1:
        return 1
    else:
        return 0

diabetes_df['hypertension_heart'] = diabetes_df.apply(has_both_conditions, axis=1)

print("✅ hypertension_heart created!")
print(diabetes_df['hypertension_heart'].value_counts())

✅ hypertension_heart created!
hypertension_heart
0    99084
1      916
Name: count, dtype: int64


In [41]:
# Create age squared feature
diabetes_df['age_squared'] = diabetes_df['age'] ** 2

print("✅ age_squared created!")
print(f"Average: {diabetes_df['age_squared'].mean():.2f}")

✅ age_squared created!
Average: 2261.43


In [42]:
# Create BMI squared feature
diabetes_df['bmi_squared'] = diabetes_df['bmi'] ** 2

print("✅ bmi_squared created!")
print(f"Average: {diabetes_df['bmi_squared'].mean():.2f}")

✅ bmi_squared created!
Average: 790.47


In [43]:
# See all columns we have now
print("=== Diabetes Dataset - All Columns ===")
print(f"Original columns: 9")
print(f"Total columns now: {len(diabetes_df.columns)}")
print(f"\nNew features created: {len(diabetes_df.columns) - 9}")

print("\n=== All Columns ===")
for col in diabetes_df.columns:
    print(f"  - {col}")

=== Diabetes Dataset - All Columns ===
Original columns: 9
Total columns now: 24

New features created: 15

=== All Columns ===
  - gender
  - age
  - hypertension
  - heart_disease
  - smoking_history
  - bmi
  - HbA1c_level
  - blood_glucose_level
  - diabetes
  - gender_encoded
  - age_group
  - bmi_category
  - glucose_category
  - hba1c_category
  - smoking_encoded
  - is_obese
  - is_elderly
  - high_glucose
  - age_bmi
  - glucose_bmi
  - risk_score
  - hypertension_heart
  - age_squared
  - bmi_squared


In [44]:
# Save the feature engineered datasets
heart_df.to_csv('../data/processed/tabular/heart_features.csv', index=False)
diabetes_df.to_csv('../data/processed/tabular/diabetes_features.csv', index=False)

print("✅ Heart features saved to: data/processed/tabular/heart_features.csv")
print("✅ Diabetes features saved to: data/processed/tabular/diabetes_features.csv")
print(f"\nHeart: {heart_df.shape[0]} rows, {heart_df.shape[1]} columns")
print(f"Diabetes: {diabetes_df.shape[0]} rows, {diabetes_df.shape[1]} columns")

✅ Heart features saved to: data/processed/tabular/heart_features.csv
✅ Diabetes features saved to: data/processed/tabular/diabetes_features.csv

Heart: 920 rows, 31 columns
Diabetes: 100000 rows, 24 columns
